### First Query

In [ ]:
WITH matchups AS (
SELECT g.win_team_id
  , g.lose_team_id
  , (SELECT win_masc.tax_family FROM `basketball.mascots` win_masc WHERE win_masc.id = g.win_team_id) AS tax_family_winner
  , (SELECT win_masc.tax_genus FROM `basketball.mascots` win_masc WHERE win_masc.id = g.win_team_id) AS tax_genus_winner
  , (SELECT lose_masc.tax_family FROM `basketball.mascots` lose_masc WHERE lose_masc.id = g.lose_team_id) AS tax_family_loser
  , (SELECT lose_masc.tax_genus FROM `basketball.mascots` lose_masc WHERE lose_masc.id = g.lose_team_id) AS tax_genus_loser
FROM `bigquery-public-data.ncaa_basketball.mbb_historical_tournament_games` g
)
SELECT
  SUM(IF(tax_family_winner = "Felidae" AND tax_genus_loser = "Canis", 1, 0)) AS num_cat_wins,
  SUM(IF(tax_genus_winner = "Canis" AND tax_family_loser = "Felidae", 1, 0)) AS num_dog_wins
FROM matchups;

## Semi-Structured Queries

### Array

In [ ]:
SELECT ARRAY<STRING> 
['raspberry', 'blackberry', 'strawberry', 'cherry'] 
AS fruit_array;

#### OFFSET

In [ ]:
WITH fruits AS(
  SELECT ARRAY<STRING> 
  ['raspberry', 'blackberry', 'strawberry', 'cherry'] 
  AS fruit_array
)
SELECT fruit_array[OFFSET(2)]
AS zero_indexed
FROM fruits;

#### ORDINAL

In [ ]:
WITH fruits AS(
  SELECT ARRAY<STRING> 
  ['raspberry', 'blackberry', 'strawberry', 'cherry'] 
  AS fruit_array
)
SELECT fruit_array[ORDINAL(2)]
AS zero_indexed
FROM fruits;

#### OUTSIDE INDEX

In [ ]:
WITH fruits AS(
  SELECT ARRAY<STRING> 
  ['raspberry', 'blackberry', 'strawberry', 'cherry'] 
  AS fruit_array
)
SELECT fruit_array[ORDINAL(999)]
AS zero_indexed
FROM fruits;

#### Array Length

In [ ]:
WITH fruits AS(
  SELECT ARRAY<STRING> 
  ['raspberry', 'blackberry', 'strawberry', 'cherry'] 
  AS fruit_array
)
SELECT ARRAY_LENGTH(fruit_array)
AS array_size
FROM fruits;

#### UNFLATTEN ARRAY

In [ ]:
SELECT 
['apple', 'pear', 'plum'] as item, 
'Jacob' AS customer;

#### Flatten Array: UNNEST

In [ ]:
SELECT items, customer_name
FROM 
  UNNEST(['apple', 'pear', 'peach']) AS ITEMS
CROSS JOIN
(SELECT 'Jacob' AS customer_name);

#### Get order of the array element

In [ ]:
SELECT index, items
FROM 
  UNNEST(['apple', 'pear', 'peach']) AS ITEMS
WITH OFFSET AS INDEX
ORDER BY INDEX;

#### Create Array

In [ ]:
SELECT ARRAY(
    SELECT 'apple' AS fruit UNION ALL
    SELECT 'pear' AS fruit UNION ALL
    SELECT 'banana' AS fruit 
)
AS fruit_basket;

#### Create Array with order

In [ ]:
SELECT ARRAY(
    SELECT 'apple' AS fruit UNION ALL
    SELECT 'pear' AS fruit UNION ALL
    SELECT 'banana' AS fruit 
    ORDER BY fruit
)
AS fruit_basket;

#### Filter Data inside array items

In [ ]:
WITH groceries AS 
(SELECT ['apple', 'pear', 'banana'] AS list
UNION ALL SELECT ['carrot', 'apple'] AS list
UNION ALL SELECT ['water', 'wine'] AS list)

SELECT ARRAY(
  SELECT list_items
  FROM UNNEST(list) as list_items
  WHERE 'apple' IN UNNEST(list)
)
FROM groceries;

### Struct: the container

In [ ]:
SELECT STRUCT<int64, STRING>(35, 'Jacob');

In [ ]:
SELECT STRUCT<int64, STRING>(35, 'Jacob')
AS CUSTOMERS;

#### Struct with array

In [ ]:
SELECT STRUCT(35 AS age, 'Jacob' AS name, ['apple', 'pear', 'peach'] AS items)
AS customers;

In [ ]:
SELECT ARRAY(
  SELECT STRUCT(35 AS age, 'Jacob' AS name, ['apple', 'pear', 'peach'] AS items)
  UNION ALL
  SELECT STRUCT(33 AS age, 'Miranda' AS name, ['water', 'pineapple', 'ice cream'] AS items)
)

### Nested Fields

In [ ]:
[
  {"name": "id", "type": "INTEGER", "mode": "REQUIRED"},
  {"name": "time", "type": "TIMESTAMP", "mode": "REQUIRED"},
  {"name": "customer", "type": "RECORD", "fields":
    [
      {"name": "id", "type": "INTEGER", "mode": "REQUIRED"},
      {"name": "name",  "type": "STRING", "mode": "REQUIRED"},
      {"name": "location",  "type": "STRING"}
    ]
  }
]

In [ ]:
WITH transactions AS
(
  SELECT 
    1000001 AS id,	
    TIMESTAMP('2017-12-18 15:02:00') AS time,	
    STRUCT(65401 AS id,	'John Doe' AS name,	'Faraway' AS location) AS customer
  UNION ALL 
  SELECT 
    1000002, 
    TIMESTAMP('2017-12-16 11:34:00'), 
    STRUCT(74682, 'Jane Michaels', 'Nearland')
)
SELECT *
FROM transactions;

### Repeated Fields

In [ ]:
[
  {"name": "id", "type": "INTEGER", "mode": "REQUIRED"},
  {"name": "time", "type": "TIMESTAMP", "mode": "REQUIRED"},
  {"name": "product", "type": "RECORD", "mode": "REPEATED", "fields":
    [
      {"name": "sku", "type": "STRING", "mode": "REQUIRED"},
      {"name": "description",  "type": "STRING"},
      {"name": "quantity",  "type": "INTEGER"},
      {"name": "price",  "type": "FLOAT", "mode": "REQUIRED"}
    ]
  }
]

#### Unflatten

In [ ]:
WITH transactions AS
(
  SELECT
    1000001 AS id,
    TIMESTAMP('2017-12-18 15:02:00') AS time,
    [ 
      STRUCT('ABC123456' AS sku, 'furniture' AS description, 3 AS quantity, 36.3 AS price),
      STRUCT('TBL535522' AS sku, 'table' AS description, 6 AS quantity, 878.4 AS price),
      STRUCT('CHR762222' AS sku, 'chair' AS description, 4 AS quantity, 435.6 AS price)
    ] AS product
  UNION ALL
  SELECT
    1000002,
    TIMESTAMP('2017-12-16 11:34:00'),
    [
      STRUCT('GCH635354', 'Chair', 4, 345.7),
      STRUCT('GRD828822', 'Gardening', 2,9.5) 
    ] 
)
SELECT * from transactions;

#### Flatten

In [ ]:
WITH transactions AS
(
  SELECT
    1000001 AS id,
    TIMESTAMP('2017-12-18 15:02:00') AS time,
    [ 
      STRUCT('ABC123456' AS sku, 'furniture' AS description, 3 AS quantity, 36.3 AS price),
      STRUCT('TBL535522' AS sku, 'table' AS description, 6 AS quantity, 878.4 AS price),
      STRUCT('CHR762222' AS sku, 'chair' AS description, 4 AS quantity, 435.6 AS price)
    ] AS product
  UNION ALL
  SELECT
    1000002,
    TIMESTAMP('2017-12-16 11:34:00'),
    [
      STRUCT('GCH635354', 'Chair', 4, 345.7),
      STRUCT('GRD828822', 'Gardening', 2,9.5) 
    ] 
)
SELECT
  id, items.*
FROM transactions, 
  UNNEST(product) AS items;

### Nested Repeated Fields

In [ ]:
[
  {"name": "id", "type": "INTEGER", "mode": "REQUIRED"},
  {"name": "time", "type": "TIMESTAMP", "mode": "REQUIRED"},
  {"name": "customer", "type": "RECORD", "fields":
    [
      {"name": "id", "type": "INTEGER", "mode": "REQUIRED"},
      {"name": "name",  "type": "STRING", "mode": "REQUIRED"},
      {"name": "location",  "type": "STRING"}
    ]
  },
  {"name": "product", "type": "RECORD", "mode": "REPEATED", "fields":
      [
        {"name": "sku", "type": "STRING", "mode": "REQUIRED"},
        {"name": "description",  "type": "STRING"},
        {"name": "quantity",  "type": "INTEGER"},
        {"name": "price",  "type": "FLOAT", "mode": "REQUIRED"}
      ]
    }
]

#### Unflatten

In [ ]:
WITH transactions AS (
  SELECT 1000001 AS order_id, TIMESTAMP('2017-12-18 15:02:00') AS order_time,
          STRUCT(65401 AS id,	'John Doe' AS name,	'Faraway' AS location) AS customer, 
          [
            STRUCT('ABC123456' AS sku, 3 AS quantity, 36.3 AS price),
            STRUCT('TBL535522' AS sku, 6 AS quantity, 878.4 AS price),
            STRUCT('CHR762222' AS sku, 4 AS quantity, 435.6 AS price)
          ] AS orders
  UNION ALL 
  SELECT 1000002, TIMESTAMP('2017-12-16 11:34:00'), 
        STRUCT(74682, 'Jane Michaels', 'Nearland') AS customer,
        [
            STRUCT('GCH635354',   4,      345.7),
            STRUCT('GRD828822', 2,      9.5)
        ] AS orders
)
SELECT * from transactions;

#### Flatten

In [ ]:
WITH transactions AS (
  SELECT 1000001 AS order_id, TIMESTAMP('2017-12-18 15:02:00') AS order_time,
          STRUCT(65401 AS id,	'John Doe' AS name,	'Faraway' AS location) AS customer, 
          [
            STRUCT('ABC123456' AS sku, 3 AS quantity, 36.3 AS price),
            STRUCT('TBL535522' AS sku, 6 AS quantity, 878.4 AS price),
            STRUCT('CHR762222' AS sku, 4 AS quantity, 435.6 AS price)
          ] AS orders
  UNION ALL 
  SELECT 1000002, TIMESTAMP('2017-12-16 11:34:00'), 
        STRUCT(74682, 'Jane Michaels', 'Nearland') AS customer,
        [
            STRUCT('GCH635354',   4,      345.7),
            STRUCT('GRD828822', 2,      9.5)
        ] AS orders
)
SELECT 
  t.order_id,
  t.customer.name,
  SUM(orders.price) AS total_purchased
FROM transactions AS t, UNNEST(orders) AS orders
GROUP BY t.order_id, t.customer.name